In [1]:
import spacy
import emoji
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from textblob import TextBlob
import string
import re
nlp = spacy.load('en_core_web_sm')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def cleanResume(resumeText):

    resumeText = re.sub('httpS+s*', ' ', resumeText)  # remove URLs
    resumeText = re.sub('RT|cc', ' ', resumeText)  # remove RT and cc
    resumeText = re.sub('#S+', '', resumeText)  # remove hashtags
    resumeText = re.sub('@S+', '  ', resumeText)  # remove mentions
    resumeText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[]^_`{|}~"""), ' ', resumeText)  # remove punctuations
    resumeText = re.sub(r'[^x00-x7f]',r' ', resumeText)
    resumeText = re.sub('s+', ' ', resumeText)  # remove extra whitespace
    return resumeText.lower()

In [3]:
import pandas as pd
df = pd.read_csv('UpdatedResumeDataSet.csv')
df.head()

,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."


In [4]:
df.shape

(962, 2)

In [4]:
df['Resume'] = df['Resume'].apply(cleanResume)
df.head()

,Category,Resume
0,Data Science,skill programming language p thon panda ...
1,Data Science,education detail ma 2013 to ma 2017 b e ...
2,Data Science,area of intere t deep learning control s te...
3,Data Science,skill r p thon sap hana table...
4,Data Science,education detail mca ymcaust faridabad...


In [5]:
import numpy as np
import spacy
import pandas as pd

nlp = spacy.load("en_core_web_sm")

sentences = [[token.text for token in nlp(resume)] for resume in df['Resume']]

from gensim.models import Word2Vec

model = Word2Vec(sentences, vector_size=100, window=5, min_count=2)

def vector(resume):
    vectors = [model.wv[word] for word in resume if word in model.wv]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(100)

vectors = [vector(resume) for resume in sentences]

vector_df = pd.DataFrame(vectors, columns=[f"vector_{i+1}" for i in range(100)])

df = pd.concat([df.reset_index(drop=True), vector_df], axis=1)

In [6]:
df.head()

,Category,Resume,vector_1,vector_2,vector_3,vector_4,vector_5,vector_6,vector_7,vector_8,...,vector_91,vector_92,vector_93,vector_94,vector_95,vector_96,vector_97,vector_98,vector_99,vector_100
0,Data Science,skill programming language p thon panda ...,0.062177,0.203484,-0.080723,-0.257798,0.120933,-0.668844,0.164110,0.794601,...,0.575527,-0.212079,-0.033838,-0.317018,-0.021168,0.147359,-0.038345,0.066183,-0.073003,-0.097586
1,Data Science,education detail ma 2013 to ma 2017 b e ...,-0.025947,0.521268,-0.176957,-0.329023,0.199745,-0.579908,0.090736,0.883440,...,0.442842,-0.326493,-0.352357,-0.310260,0.011409,0.459134,0.020883,0.338816,-0.209582,-0.093936
2,Data Science,area of intere t deep learning control s te...,-0.030881,0.511267,-0.138721,-0.189995,0.139718,-0.658995,0.123058,1.008569,...,0.529053,-0.377880,-0.313503,-0.247525,0.034931,0.466463,-0.124218,0.298010,-0.188501,-0.141780
3,Data Science,skill r p thon sap hana table...,-0.057515,0.220499,-0.114009,-0.241363,0.030867,-0.757269,0.107967,0.935839,...,0.515722,-0.209278,-0.034426,-0.360466,-0.012806,0.237533,0.044265,0.072897,-0.082502,-0.055425
4,Data Science,education detail mca ymcaust faridabad...,-0.069959,0.837766,-0.230679,-0.394082,0.408772,-0.713053,-0.025377,1.361643,...,0.546856,-0.594937,-0.798930,-0.378395,-0.001742,0.774307,-0.072991,0.666024,-0.455363,-0.073160


In [7]:
x = df.drop(columns = ['Category', 'Resume'])
x

,vector_1,vector_2,vector_3,vector_4,vector_5,vector_6,vector_7,vector_8,vector_9,vector_10,...,vector_91,vector_92,vector_93,vector_94,vector_95,vector_96,vector_97,vector_98,vector_99,vector_100
0,0.062177,0.203484,-0.080723,-0.257798,0.120933,-0.668844,0.164110,0.794601,-0.167454,-0.392916,...,0.575527,-0.212079,-0.033838,-0.317018,-0.021168,0.147359,-0.038345,0.066183,-0.073003,-0.097586
1,-0.025947,0.521268,-0.176957,-0.329023,0.199745,-0.579908,0.090736,0.883440,-0.119968,-0.546598,...,0.442842,-0.326493,-0.352357,-0.310260,0.011409,0.459134,0.020883,0.338816,-0.209582,-0.093936
2,-0.030881,0.511267,-0.138721,-0.189995,0.139718,-0.658995,0.123058,1.008569,-0.216741,-0.541775,...,0.529053,-0.377880,-0.313503,-0.247525,0.034931,0.466463,-0.124218,0.298010,-0.188501,-0.141780
3,-0.057515,0.220499,-0.114009,-0.241363,0.030867,-0.757269,0.107967,0.935839,0.038988,-0.575123,...,0.515722,-0.209278,-0.034426,-0.360466,-0.012806,0.237533,0.044265,0.072897,-0.082502,-0.055425
4,-0.069959,0.837766,-0.230679,-0.394082,0.408772,-0.713053,-0.025377,1.361643,-0.044788,-1.050426,...,0.546856,-0.594937,-0.798930,-0.378395,-0.001742,0.774307,-0.072991,0.666024,-0.455363,-0.073160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
957,0.036517,0.444694,-0.047845,-0.302497,-0.174655,-0.625670,0.205286,0.706596,0.149326,-0.387053,...,0.304155,-0.213393,-0.106451,-0.177794,0.109656,0.252430,0.028099,-0.016264,0.039941,-0.310642
958,-0.019154,0.434860,-0.046003,-0.248563,-0.086072,-0.787971,0.159659,0.865354,0.287308,-0.415475,...,0.307632,-0.059644,0.080763,-0.383783,0.233640,0.191714,0.154625,0.060250,0.067462,-0.306777
959,-0.034457,0.405866,0.070899,-0.157792,-0.289663,-0.708159,0.276361,0.660493,0.037118,-0.419878,...,0.263004,-0.008541,-0.166019,-0.169960,0.271616,0.214730,-0.070832,-0.092030,0.162269,-0.280136
960,-0.177022,0.713080,-0.066428,0.161868,0.361387,-0.414376,0.091840,1.178915,0.055584,-0.379012,...,0.466630,-0.269452,-0.353851,-0.211202,-0.041036,0.771217,0.030044,0.506261,-0.305640,-0.197324


In [8]:
y = df['Category']
y

0      Data Science
1      Data Science
2      Data Science
3      Data Science
4      Data Science
           ...     
957         Testing
958         Testing
959         Testing
960         Testing
961         Testing
Name: Category, Length: 962, dtype: object

In [9]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, shuffle = True)

In [11]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

grid_search = GridSearchCV(
    estimator=KNeighborsClassifier(),
    param_grid=param_grid,
    cv=5,
    verbose=1,
)

grid_search.fit(x_train, y_train)

best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

print(f"Best : {best_params}")

y_pred = best_model.predict(x_test)


Fitting 5 folds for each of 40 candidates, totalling 200 fits
Best : {'metric': 'euclidean', 'n_neighbors': 1, 'weights': 'distance'}


In [12]:
#  best_model
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9948186528497409

In [13]:
best_model.predict( np.array((x.T[800])).reshape(1, 100))

array(['ETL Developer'], dtype=object)

In [18]:
model.save("word2vec_model.bin")

In [19]:
import pickle
with open('resume_model.pkl', 'wb') as f :
    pickle.dump(best_model, f)